# AI Computer Assignment 2 (Minimax Algorithms)
Mohammad Saadati - 
_810198410_

## Introduction
The aim of this project is to implement **Connect 4** game with Artificial Intelligence opponent using *minimax* algorithm with and without *alpha-beta* pruning.

### Import Libraries
In this part, some of the necessary libraries were imported in order to use their helpful functions.

In [1]:
from random import random
from random import shuffle
import copy
from time import time
import pandas as pd

### Defining Constants
In this part, constant values are defined in order to make the code more readable and more flexible to change.

In [2]:
DEPTH = 1
IS_PRUNE_ON = False

## Connect4 Class

In [3]:
class ConnectSin:
    YOU = 1
    CPU = -1
    EMPTY = 0
    DRAW = 0
    __CONNECT_NUMBER = 4
    board = None

    def __init__(self, board_size=(6, 7), silent=False):
        """
        The main class for the connect4 game

        Inputs
        ----------
        board_size : a tuple representing the board size in format: (rows, columns)
        silent     : whether the game prints outputs or not
        """
        assert len(board_size) == 2, "board size should be a 1*2 tuple"
        assert board_size[0] > 4 and board_size[1] > 4, "board size should be at least 5*5"

        self.columns = board_size[1]
        self.rows = board_size[0]
        self.silent = silent
        self.board_size = self.rows * self.columns
        
        self.total_visited_nodes = 0
        self.last_empty_space = 0

    def run(self, starter=None):
        """
        runs the game!

        Inputs
        ----------
        starter : either -1,1 or None. -1 if cpu starts the game, 1 if you start the game. None if you want the starter
            to be assigned randomly 

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that the game has drawn
        """
        if (not starter):
            starter = self.__get_random_starter()
        assert starter in [self.YOU, self.CPU], "starter value can only be 1,-1 or None"
        
        self.__init_board()
        turns_played = 0
        current_player = starter
        while(turns_played < self.board_size):
            
            if (current_player == self.YOU):
                self.__print_board()
                player_input = self.get_your_input()
            elif (current_player == self.CPU):
                player_input = self.__get_cpu_input()
            else:
                raise Exception("A problem has happend! contact no one, there is no fix!")
            if (not self.register_input(player_input, current_player)):
                self.__print("this move is invalid!")
                continue
            current_player = self.__change_turn(current_player)
            potential_winner = self.check_for_winners()
            turns_played += 1
            if (potential_winner != 0):
                self.__print_board()
                self.__print_winner_message(potential_winner)
                return potential_winner
        self.__print_board()
        self.__print("The game has ended in a draw!")
        return self.DRAW

    
    def cal_score(self, connect, piece):
        score = 0
        
        # Score Horizontal
        for i in range(self.rows):
            for j in range(self.columns - connect + 1):
                is_same = False
                for k in range(connect):
                    if self.board[i][j + k] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
                    
        # Score Vertical
        for i in range(self.rows - connect + 1):
            for j in range(self.columns):
                is_same = False
                for k in range(connect):
                    if self.board[i + k][j] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
        
        # Score posiive sloped diagonal
        for i in range(self.rows - connect + 1):
            for j in range(self.columns - connect + 1):
                is_same = False
                for k in range(connect):
                    if self.board[i + k][j + k] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
        
        # Score negiive sloped diagonal
        for i in range(self.rows - connect + 1):
            for j in range(self.columns - connect + 1):
                is_same = False
                for k in range(connect):
                    if self.board[i + k][self.columns - 1 - (j + k)] == piece:
                        is_same = True
                    else:
                        is_same = False
                        break
                if is_same:
                    score += 1
                    
        return score
                    
    def evaluate(self):
        winner = self.check_for_winners()
        if winner == self.YOU:
            return float('inf')
        elif winner == self.CPU:
            return float('-inf')
    
        total_score = 0
        for i in range(2, self.__CONNECT_NUMBER):
            total_score += (6*(10**((i-2)))*self.cal_score(i, self.YOU) - 6*(10**((i-2)))*self.cal_score(i, self.CPU))
        return total_score   
    
    def minimax(self, _move, depth, max_player, alpha, beta):
        self.total_visited_nodes += 1
        
        if not self.silent:
            print(self.__print_board())
            print(self.evaluate())
            
        best_move = None
        moves = self.get_possible_moves()
        if self.check_for_winners() != 0 or depth == 0 or len(moves) == 0 :
            return self.evaluate(), _move
        
        if IS_PRUNE_ON:
            shuffle(moves)       

        if max_player:
            max_evaluation = float('-inf')
            for move in moves:
                self.register_input(move, self.YOU)
                _last_empty_space = self.last_empty_space
                evaluation = self.minimax(move, depth - 1, False, alpha, beta)[0]
                max_evaluation = max(evaluation, max_evaluation)
                self.board[_last_empty_space][move - 1] = 0
                if evaluation == max_evaluation:
                    best_move = move
                    alpha = max(alpha, max_evaluation)
                    if IS_PRUNE_ON and alpha >= beta:
                        break
            return max_evaluation, best_move
        else:
            min_evaluation = float('inf')
            for move in moves:
                self.register_input(move, self.CPU)
                _last_empty_space = self.last_empty_space
                evaluation = self.minimax(move, depth - 1, True, alpha, beta)[0]
                min_evaluation = min(evaluation, min_evaluation)
                self.board[_last_empty_space][move - 1] = 0
                if evaluation == min_evaluation:
                    best_move = move
                    beta = min(beta, min_evaluation)
                    if IS_PRUNE_ON and alpha >= beta:
                        break
            return min_evaluation, best_move
    
    def get_your_input(self):
        """
        gets your input

        Output
        ----------
        (int) an integer between 1 and column count. the column to put a piece in
        """
        return self.minimax(0, DEPTH, True, float('-inf'), float('inf'))[1]۳

    def check_for_winners(self):
        """
        checks if anyone has won in this position

        Output
        ----------
        (int) either 1,0,-1. 1 meaning you have won, -1 meaning the player has won and 0 means that nothing has happened
        """
        have_you_won = self.check_if_player_has_won(self.YOU)
        if have_you_won:
            return self.YOU
        has_cpu_won = self.check_if_player_has_won(self.CPU)
        if has_cpu_won:
            return self.CPU
        return self.EMPTY

    def check_if_player_has_won(self, player_id):
        """
        checks if player with player_id has won

        Inputs
        ----------
        player_id : the id for the player to check

        Output
        ----------
        (boolean) true if the player has won in this position
        """
        return (
            self.__has_player_won_diagonally(player_id)
            or self.__has_player_won_horizentally(player_id)
            or self.__has_player_won_vertically(player_id)
        )
    
    def is_move_valid(self, move):
        """
        checks if this move can be played

        Inputs
        ----------
        move : the column to place a piece in, in range [1, column count]

        Output
        ----------
        (boolean) true if the move can be played
        """
        if (move < 1 or move > self.columns):
            return False
        column_index = move - 1
        return self.board[0][column_index] == 0
    
    def get_possible_moves(self):
        """
        returns a list of possible moves for the next move

        Output
        ----------
        (list) a list of numbers of columns that a piece can be placed in
        """
        possible_moves = []
        for i in range(self.columns):
            move = i + 1
            if (self.is_move_valid(move)):
                possible_moves.append(move)
        return possible_moves
    
    def register_input(self, player_input, current_player):
        """
        registers move to board, remember that this function changes the board

        Inputs
        ----------
        player_input : the column to place a piece in, in range [1, column count]
        current_player: ID of the current player, either self.YOU or self.CPU

        """
        if (not self.is_move_valid(player_input)):
            return False
        self.__drop_piece_in_column(player_input, current_player)
        return True

    def __init_board(self):
        self.board = []
        for i in range(self.rows):
            self.board.append([self.EMPTY] * self.columns)

    def __print(self, message: str):
        if not self.silent:
            print(message)

    def __has_player_won_horizentally(self, player_id):
        for i in range(self.rows):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_vertically(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __has_player_won_diagonally(self, player_id):
        for i in range(self.rows - self.__CONNECT_NUMBER + 1):
            for j in range(self.columns - self.__CONNECT_NUMBER + 1):
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
                has_won = True
                for x in range(self.__CONNECT_NUMBER):
                    if self.board[i + self.__CONNECT_NUMBER - 1 - x][j + x] != player_id:
                        has_won = False
                        break
                if has_won:
                    return True
        return False

    def __get_random_starter(self):
        players = [self.YOU, self.CPU]
        return players[int(random() * len(players))]
    
    def __get_cpu_input(self):
        """
        This is where clean code goes to die.
        """
        bb = copy.deepcopy(self.board)
        pm = self.get_possible_moves()
        for m in pm:
            self.register_input(m, self.CPU)
            if (self.check_if_player_has_won(self.CPU)):
                self.board = bb
                return m
            self.board = copy.deepcopy(bb)
        if (self.is_move_valid((self.columns // 2) + 1)):
            c = 0
            cl = (self.columns // 2) + 1
            for x in range(self.rows):
                if (self.board[x][cl] == self.CPU):
                    c += 1
            if (random() < 0.65):
                return cl
        return pm[int(random() * len(pm))]
    
    def __drop_piece_in_column(self, move, current_player):
        last_empty_space = 0
        column_index = move - 1
        for i in range(self.rows):
            if (self.board[i][column_index] == 0):
                last_empty_space = i
        self.board[last_empty_space][column_index] = current_player
        self.last_empty_space = last_empty_space
        return True
        
    def __print_winner_message(self, winner):
        if (winner == self.YOU):
            self.__print("congrats! you have won!")
        else:
            self.__print("gg. CPU has won!")
    
    def __change_turn(self, turn):
        if (turn == self.YOU): 
            return self.CPU
        else:
            return self.YOU

    def __print_board(self):
        if (self.silent): return
        print("Y: you, C: CPU")
        for i in range(self.rows):
            for j in range(self.columns):
                house_char = "O"
                if (self.board[i][j] == self.YOU):
                    house_char = "Y"
                elif (self.board[i][j] == self.CPU):
                    house_char = "C"
                    
                print(f"{house_char}", end=" ")
            print()


## Heuristic 
The heuristic function is defined to evaluate each of the states in a function called ‍‍‍`evaluate`. In this function, the number of consecutive double pieces, the number of consecutive triple pieces and ... are counted for each player, and a coefficient is assigned to each one, then these values are added together as a *YOU* score and *CPU* score. Finally, the result of subtracting the *YOU* and *CPU* score is returned as the value of `evaluate` function.

In [10]:
reslut_data = {'Board Size':[],
               'Number of Runs':[],
                'Depth':[],
                'Prune':[],
                'Win Chance':[],
                'Average Time(s)':[],
                'Average Visited Nodes':[]}

def update_result_data(borad, run, depth, is_prune_on, win_chanse, average_time, average_total_visited_nodes):
    x = reslut_data['Board Size']
    x.append(str(borad[0]) + " * " + str(borad[1]))
    reslut_data['Board Size'] = x
    
    x = reslut_data['Number of Runs']
    x.append(run)
    reslut_data['Number of Runs'] = x
    
    x = reslut_data['Depth']
    x.append(depth)
    reslut_data['Depth'] = x
    
    x = reslut_data['Prune']
    x.append(is_prune_on)
    reslut_data['Prune'] = x
    
    x = reslut_data['Win Chance']
    x.append(win_chanse)
    reslut_data['Win Chance'] = x
    
    x = reslut_data['Average Time(s)']
    x.append(average_time)
    reslut_data['Average Time(s)'] = x
    
    x = reslut_data['Average Visited Nodes']
    x.append(average_total_visited_nodes)
    reslut_data['Average Visited Nodes'] = x  

def run_game_with_params(board, run, depth, is_prune_on):
    global DEPTH
    global IS_PRUNE_ON
    DEPTH = depth
    IS_PRUNE_ON = is_prune_on
    
    win_chanse = 0
    average_time = 0
    average_total_visited_nodes = 0
    
    for i in range(run):
        game = ConnectSin(board_size=board,silent=True)
        start_time = time()
        game.run()
        finish_time = time()
        
        average_total_visited_nodes += game.total_visited_nodes
        average_time += (finish_time - start_time)
        if game.check_for_winners() == game.YOU:
            win_chanse += 1
        
    average_time /= run
    average_total_visited_nodes /= run
    win_chanse /= run
    
    update_result_data(board, run, depth, is_prune_on, win_chanse, average_time, average_total_visited_nodes)
    print(board, run, depth, is_prune_on, win_chanse, average_time, average_total_visited_nodes
          , int((time() - s_total_time)/60), "min and", '%.2f'%((time() - s_total_time)%60), "secs")
    
def run_game(board, run):
    run_game_with_params(board, run, 1, False)
    run_game_with_params(board, run, 3, False)
    run_game_with_params(board, run, 5, False)
    
    run_game_with_params(board, run, 1, True)
    run_game_with_params(board, run, 3, True)
    run_game_with_params(board, run, 5, True)
    run_game_with_params(board, run, 7, True)

board_sizes_to_check = [# (6,7), 
                        # (7,8), 
                        (7,10)]
number_of_runs = [# 100, 
                  # 20, 
                  20]

# print("Result for", NUMBER_OF_RUN, "runs ....")
print("Running ....")
s_total_time = time()
for board, cur_run in zip(board_sizes_to_check, number_of_runs):
    run_game(board, cur_run)
    print(board, cur_run, int((time() - s_total_time)/60), "min and", '%.2f'%((time() - s_total_time)%60), "secs")

print("FINISH", int((time() - s_total_time)/60), "min and", '%.2f'%((time() - s_total_time)%60), "secs")
result_df = pd.DataFrame(reslut_data)
result_df

Running ....
(7, 10) 20 1 False 0.7 0.014624059200286865 44.0 0 min and 0.29 secs
(7, 10) 20 3 False 1.0 1.2159884095191955 4551.45 0 min and 24.61 secs
(7, 10) 20 5 False 1.0 130.7345956325531 467088.4 43 min and 59.31 secs
(7, 10) 20 1 True 0.65 0.014683771133422851 41.45 43 min and 59.60 secs
(7, 10) 20 3 True 0.85 0.3494519829750061 1307.95 44 min and 6.59 secs
(7, 10) 20 5 True 0.65 7.995219445228576 31232.1 46 min and 46.50 secs
(7, 10) 20 7 True 0.6 243.40643541812898 959643.45 127 min and 54.63 secs
(7, 10) 20 127 min and 54.63 secs
FINISH 127 min and 54.63 secs


,Board Size,Number of Runs,Depth,Prune,Win Chance,Average Time(s),Average Visited Nodes
0,7 * 10,20,1,False,0.70,0.014624,44.00
1,7 * 10,20,3,False,1.00,1.215988,4551.45
2,7 * 10,20,5,False,1.00,130.734596,467088.40
3,7 * 10,20,1,True,0.65,0.014684,41.45
4,7 * 10,20,3,True,0.85,0.349452,1307.95
5,7 * 10,20,5,True,0.65,7.995219,31232.10
6,7 * 10,20,7,True,0.60,243.406435,959643.45


In [11]:
result_df = pd.DataFrame(reslut_data)
result_df.to_csv('result_of_runs_15to21.csv')

## Questions 
### 1.
Heuristic function is used in Minimax for evaluation of the current situation of the game. The final decision made by Minimax largely depends on how well the heuristic function is. Therefore, designing a reasonable heuristic function is paramount. To evaluate the current situation of the game, the heuristic function firstly looks for different features on the board and then gives them proper values. Finally, the heuristic function returns a summation of all the values of features on the chess board. 
### 2.
According to the obtained results, it can be seen that increasing the depth, increases the chances of wins, the number of nodes visited and the execution time. We find out that as depth of searching keeps increasing, a heuristic has better functionality. Moreover, we also find out that as number of features rises up, a heuristic becomes more optimal. Besides, if we increase the search depth of a relatively weaker heuristic with much less number of features, that “weaker” heuristic can beat its opponent with more features.
### 3.
The sequence of the moves that we make is importent in Alpha-beta pruning. 
While finding the minimum if the min value is found at the first move then other branches will be omitted meaning that many parts of the tree won't be searched therefore the search will be much quicker. 
This idea is also correct for finding the maximum value if we find maximum value at the first move then more branches will be omitted and therefore the search would be faster. 
So we have to arrange the sequence in a way that we find min and max values as soon as possible and since we know what each of the moves can do we can arrange them in a way that is relatively better also the value that we get is based on evaluate function so we have arrange based on this value. When it's player's turn we are trying to choose the max value so based on the evaluation function the opponent's part isn't changing so we have to make the move that maximizes the player's part. When it's opponents turn the same process will happen but instead, player's part isn't changing and we want to maxmize opponent's part so at the end we have the minimum value from evaluate function.